참고문헌: PyTorch로 시작하는 딥 러닝 입문, 파이썬 딥러닝 파이토치 (이경택, 방성수, 안상준 지음), 정보문화사

# 11. 다대다 RNN을 이용한 텍스트 생성


## 01. 문자 단위 RNN(Char RNN)

이번 챕터에서는 모든 시점의 입력에 대해서 모든 시점에 대해서 출력을 하는 다대다 RNN을 구현해봅시다. 다대다 RNN은 대표적으로 품사 태깅, 개체명 인식 등에서 사용됩니다.

## 1. 문자 단위 RNN(Char RNN)


RNN의 입출력의 단위가 단어 레벨(word-level)이 아니라 문자 레벨(character-level)로 하여 RNN을 구현한다면, 이를 문자 단위 RNN이라고 합니다. RNN 구조 자체가 달라진 것은 아니고, 입, 출력의 단위가 문자로 바뀌었을 뿐입니다. 문자 단위 RNN을 다대다 구조로 구현해봅시다.

우선 필요한 도구들을 임포트합니다.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

## 1. 훈련 데이터 전처리하기

여기서는 문자 시퀀스 apple을 입력받으면 pple!를 출력하는 RNN을 구현해볼 겁니다. 이렇게 구현하는 어떤 의미가 있지는 않습니다. 그저 RNN의 동작을 이해하기 위한 목적입니다.

입력 데이터와 레이블 데이터에 대해서 문자 집합(voabulary)을 만듭니다. 여기서 문자 집합은 중복을 제거한 문자들의 집합입니다.

In [ ]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

문자 집합의 크기 : 5


현재 문자 집합에는 총 5개의 문자가 있습니다. !, a, e, l, p입니다. 이제 하이퍼파라미터를 정의해줍니다. 이때 입력은 원-핫 벡터를 사용할 것이므로 입력의 크기는 문자 집합의 크기여야만 합니다.

In [ ]:
input_size = vocab_size # 입력의 크기는 문자 집합의 크기
hidden_size = 5
output_size = 5
learning_rate = 0.1

이제 문자 집합에 고유한 정수를 부여합니다.

In [ ]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 문자에 고유한 정수 인덱스 부여
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


!은 0, a는 1, e는 2, l은 3, p는 4가 부여되었습니다. 나중에 예측 결과를 다시 문자 시퀀스로 보기위해서 반대로 정수로부터 문자를 얻을 수 있는 index_to_char을 만듭니다.

In [ ]:
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


이제 입력 데이터와 레이블 데이터의 각 문자들을 정수로 맵핑합니다.

In [ ]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


파이토치의 nn.RNN()은 기본적으로 3차원 텐서를 입력받습니다. 그렇기 때문에 배치 차원을 추가해줍니다.

In [ ]:
# 배치 차원 추가
# 텐서 연산인 unsqueeze(0)를 통해 해결할 수도 있었음.
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


입력 시퀀스의 각 문자들을 원-핫 벡터로 바꿔줍니다.

In [ ]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


입력 데이터와 레이블 데이터를 텐서로 바꿔줍니다.

In [ ]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """Entry point for launching an IPython kernel.


이제 각 텐서의 크기를 확인해봅시다.

In [ ]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


## 2. 모델 구현하기

이제 RNN 모델을 구현해봅시다. 아래에서 fc는 완전 연결층(fully-connected layer)을 의미하며 출력층으로 사용됩니다.

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # RNN 셀 구현
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현

    def forward(self, x): # 구현한 RNN 셀과 출력층을 연결
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

클래스로 정의한 모델을 net에 저장합니다.

In [ ]:
net = Net(input_size, hidden_size, output_size)

이제 입력된 모델에 입력을 넣어서 출력의 크기를 확인해봅시다.

In [ ]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([1, 5, 5])


(1, 5, 5)의 크기를 가지는데 각각 배치 차원, 시점(timesteps), 출력의 크기입니다. 나중에 정확도를 측정할 때는 이를 모두 펼쳐서 계산하게 되는데, 이때는 view를 사용하여 배치 차원과 시점 차원을 하나로 만듭니다.

In [ ]:
print(outputs.view(-1, input_size).shape) # 2차원 텐서로 변환

torch.Size([5, 5])


차원이 (5, 5)가 된 것을 볼 수 있습니다. 이제 레이블 데이터의 크기를 다시 복습봅시다.

In [ ]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


레이블 데이터는 (1, 5)의 크기를 가지는데, 마찬가지로 나중에 정확도를 측정할 때는 이걸 펼쳐서 계산할 예정입니다. 이 경우 (5)의 크기를 가지게 됩니다. 이제 옵티마이저와 손실 함수를 정의합니다.

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

총 100번의 에포크를 학습합니다.

In [ ]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드.
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.7093403339385986 prediction:  [[2 3 3 2 3]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ellel
1 loss:  1.4315253496170044 prediction:  [[3 3 3 3 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  llll!
2 loss:  1.2190470695495605 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
3 loss:  0.9982198476791382 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
4 loss:  0.8037036061286926 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
5 loss:  0.6518546342849731 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
6 loss:  0.5290197134017944 prediction:  [[4 4 4 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pppe!
7 loss:  0.4172944128513336 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
8 loss:  0.32096797227859497 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.24315819144248962 prediction:  [[4 4 3 2 0]

## 02. 문자 단위 RNN(Char RNN) - 더 많은 데이터


이번 챕터에서는 더 많은 데이터 문자 단위 RNN을 구현합니다.

### 2. 문자 단위 RNN(Char RNN)


우선 필요한 도구들을 임포트합니다.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

다음과 같이 임의의 샘플을 만듭니다.

### 1. 훈련 데이터 전처리하기


In [ ]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

문자 집합을 생성하고, 각 문자에 고유한 정수를 부여합니다.

In [ ]:
char_set = list(set(sentence)) # 중복을 제거한 문자 집합 생성
char_dic = {c: i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩

In [ ]:
print(char_dic) # 공백도 여기서는 하나의 원소

{'a': 0, ' ': 1, 'c': 2, 'i': 3, 'b': 4, "'": 5, 'm': 6, 'y': 7, 't': 8, 'n': 9, ',': 10, 'h': 11, 'g': 12, 'o': 13, 'k': 14, 'u': 15, 'p': 16, 'w': 17, 's': 18, '.': 19, 'f': 20, 'e': 21, 'r': 22, 'd': 23, 'l': 24}


각 문자에 정수가 부여되었으며, 총 25개의 문자가 존재합니다. 문자 집합의 크기를 확인해봅시다.

In [ ]:
dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))

문자 집합의 크기 : 25


문자 집합의 크기는 25이며, 입력을 원-핫 벡터로 사용할 것이므로 이는 매 시점마다 들어갈 입력의 크기이기도 합니다. 이제 하이퍼파라미터를 설정합니다. hidden_size(은닉 상태의 크기)를 입력의 크기와 동일하게 줬는데, 이는 사용자의 선택으로 다른 값을 줘도 무방합니다.

그리고 sequence_length라는 변수를 선언했는데, 우리가 앞서 만든 샘플을 10개 단위로 끊어서 샘플을 만들 예정이기 때문입니다. 이는 뒤에서 더 자세히 보겠습니다.

In [ ]:
# 하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10  # 임의 숫자 지정
learning_rate = 0.1

다음은 임의로 지정한 sequence_length 값인 10의 단위로 샘플들을 잘라서 데이터를 만드는 모습을 보여줍니다.

In [ ]:
# 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])  # x str to index
    y_data.append([char_dic[c] for c in y_str])  # y str to index

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

총 170개의 샘플이 생성되었습니다. 그리고 각 샘플의 각 문자들은 고유한 정수로 인코딩이 된 상태입니다. 첫번째 샘플의 입력 데이터와 레이블 데이터를 출력해봅시다.

In [ ]:
print(x_data[0])
print(y_data[0])

[3, 20, 1, 7, 13, 15, 1, 17, 0, 9]
[20, 1, 7, 13, 15, 1, 17, 0, 9, 8]


한 칸씩 쉬프트 된 시퀀스가 정상적으로 출력되는 것을 볼 수 있습니다. 이제 입력 시퀀스에 대해서 원-핫 인코딩을 수행하고, 입력 데이터와 레이블 데이터를 텐서로 변환합니다.

In [ ]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # x 데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

이제 훈련 데이터와 레이블 데이터의 크기를 확인해봅시다.

In [ ]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])


원-핫 인코딩 된 결과를 보기 위해서 첫번째 샘플만 출력해봅시다.

In [ ]:
print(X[0])

tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

레이블 데이터의 첫번째 샘플도 출력해봅시다.

In [ ]:
print(Y[0])

tensor([20,  1,  7, 13, 15,  1, 17,  0,  9,  8])


### 2. 모델 구현하기


모델은 앞서 실습한 문자 단위 RNN 챕터와 거의 동일합니다. 다만 이번에는 은닉층을 두 개 쌓을 겁니다.

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size는 dic_size와 같음.
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [ ]:
net = Net(dic_size, hidden_size, 2) # 이번에는 층을 두 개 쌓습니다.

nn.RNN() 안에 num_layers라는 인자를 사용합니다. 이는 은닉층을 몇 개 쌓을 것인지를 의미합니다. 모델 선언 시 layers라는 인자에 2를 전달하여 은닉층을 두 개 쌓습니다. 비용 함수와 옵티마이저를 선언합니다.

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

이제 모델에 입력을 넣어서 출력의 크기를 확인해봅시다.

In [ ]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([170, 10, 25])


(170, 10, 25)의 크기를 가지는데 각각 배치 차원, 시점(timesteps), 출력의 크기입니다. 나중에 정확도를 측정할 때는 이를 모두 펼쳐서 계산하게 되는데, 이때는 view를 사용하여 배치 차원과 시점 차원을 하나로 만듭니다.

In [ ]:
print(outputs.view(-1, dic_size).shape) # 2차원 텐서로 변환.

torch.Size([1700, 25])


차원이 (1700, 25)가 된 것을 볼 수 있습니다. 이제 레이블 데이터의 크기를 다시 복습봅시다.

In [ ]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


레이블 데이터는 (170, 10)의 크기를 가지는데, 마찬가지로 나중에 정확도를 측정할 때는 이걸 펼쳐서 계산할 예정입니다. 이 경우 (1700)의 크기를 가지게 됩니다. 이제 옵티마이저와 손실 함수를 정의합니다.

In [ ]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # results의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오지만
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]

    print(predict_str)

kummmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm
e                                                                                                                                                                                  
ttt ttt ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
                                                                                                                                                                                   
m .....k. h hhohh h hlhhehoh hhhohoho hlhsllhhhh the o hhhhhhthh ohhh hh   sh thh ohehohhhhlh ohohe hh hhhohhaohthe thhhho hhh h  hhh  hh hhhhohh hhh thhhh sho i hhhehhhh  hhhhhhl
r .tililmrrrrrr rprrrrrrrrrrr mrrrrrrrrmurmrrrrrrr rrmrrrrrrrrrrrrrrrrrrrrrmrrrrmrr mrmrrmrrrrrrr rr

처음에는 이상한 예측을 하지만 마지막 에포크에서는 꽤 정확한 문자을 생성하는 것을 볼 수 있습니다.

# 03. 단어 단위 RNN - 임베딩 사용

이번 챕터에서는 문자 단위가 아니라 RNN의 입력 단위를 단어 단위로 사용합니다. 그리고 단어 단위를 사용함에 따라서 Pytorch에서 제공하는 임베딩 층(embedding layer)를 사용하겠습니다.

### 1. 훈련 데이터 전처리하기


우선 실습을 위해 필요한 도구들을 임포트합니다.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

실습을 위해 임의의 문장을 만듭니다.

In [ ]:
sentence = "Repeat is the best medicine for memory".split()

우리가 만들 RNN은 'Repeat is the best medicine for'을 입력받으면 'is the best medicine for memory'를 출력하는 RNN입니다. 위의 임의의 문장으로부터 단어장(vocabulary)을 만듭니다.

In [ ]:
vocab = list(set(sentence))
print(vocab)

['medicine', 'Repeat', 'best', 'for', 'is', 'memory', 'the']


이제 단어장의 단어에 고유한 정수 인덱스를 부여합니다. 그리고 그와 동시에 모르는 단어를 의미하는 UNK 토큰도 추가하겠습니다.

In [ ]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0

In [ ]:
print(word2index)

{'medicine': 1, 'Repeat': 2, 'best': 3, 'for': 4, 'is': 5, 'memory': 6, 'the': 7, '<unk>': 0}


이제 word2index가 우리가 사용할 최종 단어장인 셈입니다. word2index에 단어를 입력하면 맵핑되는 정수를 리턴합니다.

In [ ]:
print(word2index['memory'])

6


단어 'memory'와 맵핑되는 정수는 2입니다. 예측 단계에서 예측한 문장을 확인하기 위해 idx2word도 만듭니다.

In [ ]:
# 수치화된 데이터를 단어로 바꾸기 위한 사전
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'medicine', 2: 'Repeat', 3: 'best', 4: 'for', 5: 'is', 6: 'memory', 7: 'the', 0: '<unk>'}


idx2word는 정수로부터 단어를 리턴하는 역할을 합니다. 정수 2를 넣어봅시다.

In [ ]:
print(index2word[2])

Repeat


정수 2와 맵핑되는 단어는 memory인 것을 확인할 수 있습니다. 이제 데이터의 각 단어를 정수로 인코딩하는 동시에, 입력 데이터와 레이블 데이터를 만드는 build_data라는 함수를 만들어보겠습니다.

In [ ]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

만들어진 함수로부터 입력 데이터와 레이블 데이터를 얻습니다.

In [ ]:
X, Y = build_data(sentence, word2index)

입력 데이터와 레이블 데이터가 정상적으로 생성되었는지 출력해봅시다.

In [ ]:
print(X)
print(Y)

tensor([[2, 5, 7, 3, 1, 4]])
tensor([[5, 7, 3, 1, 4, 6]])


### 2. 모델 구현하기

이제 모델을 설계합니다. 이전 모델들과 달라진 점은 임베딩 층을 추가했다는 점입니다. 파이토치에서는 nn.Embedding()을 사용해서 임베딩 층을 구현합니다. 임베딩층은 크게 두 가지 인자를 받는데 첫번째 인자는 단어장의 크기이며, 두번째 인자는 임베딩 벡터의 차원입니다.

In [ ]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

이제 모델을 위해 하이퍼파라미터를 설정합니다.

In [ ]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

모델을 생성합니다.


In [ ]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

모델에 입력을 넣어서 출력을 확인해봅시다.

In [ ]:
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태이다.
output = model(X)
print(output)

tensor([[ 0.1611, -0.0612, -0.1192, -0.1996, -0.1597,  0.1927,  0.1063,  0.1457],
        [-0.4126, -0.2364, -0.2545, -0.5599, -0.2718, -0.1986,  0.1018, -0.0931],
        [-0.1189, -0.2396, -0.1202, -0.6827, -0.2776, -0.4387,  0.1492,  0.1545],
        [ 0.1076, -0.0316, -0.2023, -0.3697, -0.1470, -0.2619, -0.0283,  0.1793],
        [ 0.3077, -0.0206, -0.3697, -0.0577,  0.0934,  0.0957,  0.1668,  0.5142],
        [-0.2550,  0.0404, -0.2842, -0.2434, -0.0866,  0.0633,  0.1435,  0.1039]],
       grad_fn=<ViewBackward0>)


모델이 어떤 예측값을 내놓기는 하지만 현재 가중치는 랜덤 초기화되어 있어 의미있는 예측값은 아닙니다. 예측값의 크기를 확인해봅시다.

In [ ]:
print(output.shape)

torch.Size([6, 8])


예측값의 크기는 (6, 8)입니다. 이는 각각 (시퀀스의 길이, 은닉층의 크기)에 해당됩니다. 모델은 훈련시키기 전에 예측을 제대로 하고 있는지 예측된 정수 시퀀스를 다시 단어 시퀀스로 바꾸는 decode 함수를 만듭니다.

In [ ]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

약 200 에포크 학습합니다.

In [ ]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.0796 
Repeat is memory the the the memory

[41/201] 1.4936 
Repeat is the the medicine for memory

[81/201] 0.8560 
Repeat is the best medicine for memory

[121/201] 0.4176 
Repeat is the best medicine for memory

[161/201] 0.2118 
Repeat is the best medicine for memory

[201/201] 0.1262 
Repeat is the best medicine for memory

